In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import albumentations as A
from albumentations.pytorch import transforms, ToTensorV2

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

import json
import joblib

from sklearn.metrics import f1_score, accuracy_score

In [2]:
# Configs
config = {}
config_path = "./config/swin_all.json"
with open(config_path, 'r') as f:
    config = json.load(f)
    f.close()

In [3]:
config['DEVICE'] = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class CustomDataset(Dataset):
    def __init__(self,
                 data_path,
                 size,
                 transform=None,
                 fold=0,
                 mode="train"):
        self.csv = pd.read_csv(data_path)
        if 'kfold' in self.csv:
            if mode == "train":
                self.csv = self.csv[self.csv['kfold'] != fold]
            elif mode == "validation":
                self.csv = self.csv[self.csv['kfold'] == fold]
        
        self.path = self.csv['path'].to_list()
        if 'encoded_label' in self.csv:
            self.labels = self.csv['encoded_label'].to_list()
        self.transform = transform
        self.size = size
        self.mode = mode
    
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, idx):        
        # Image
        image = cv2.imread(self.path[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            image = self.transform(self.size)(image=image)['image']
        
        # Only test mode
        if self.mode == "test":
            return {
                'image': image
            }
        
        # Label
        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.long)
        
        return {
            'image': image,
            'label': label
        }

In [5]:
def create_validation_transforms(size):
    return A.Compose([
        A.Resize(size, size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

In [6]:
test_dataset = CustomDataset(
    data_path = config["TEST_CSV"],
    size = config["SIZE"],
    transform = create_validation_transforms,
    fold = config["FOLD"],
    mode = "test"
)
test_loader = DataLoader(
    dataset = test_dataset,
    shuffle = False,
    batch_size = config["BATCH_SIZE"],
    num_workers = config["N_WORKERS"]
)

In [7]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [8]:
with open('encoder.pickle', 'rb') as f:
    encoder = joblib.load(f)

In [9]:
def load_model(model_name = None, pretrained = True, num_classes = 88):
    return timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)

In [10]:
model = load_model(config["MODEL"], config["PRETRAINED"], config["N_CLASSES"]).to(config["DEVICE"])

f_score_avg = []
model_score_sum = 0
for fold in config["FOLD"]:
    best_model_name = f"{config['MODEL_SAVE_PREFIX']}_{fold}_best.pth"
    best_model_path = os.path.join(config['MODEL_SAVE'], best_model_name)
    model_data = torch.load(best_model_path)
    print(model_data["epoch"], model_data["score"], model_data["loss"])
    model_score_sum += model_data['score']
    model.load_state_dict(model_data['state_dict'])
    
    model.eval()

    f_score = []
    with torch.no_grad():
        pbar = tqdm(test_loader, total=len(test_loader))
        for batch in pbar:
            x = torch.tensor(batch['image'], dtype=torch.float32, device=config["DEVICE"])
            with torch.cuda.amp.autocast():
                pred = model(x)
            f_score.extend(pred.detach().cpu().numpy().tolist())
    if fold == 0:
        f_score_avg = f_score
    else:
        a = np.array(f_score_avg)
        b = np.array(f_score)
        f_score_avg = a + b

14 0.7421643212630613 0.1037438376397067


100%|██████████| 68/68 [00:15<00:00,  4.51it/s]


26 0.8163526326732292 0.08728190993025675


100%|██████████| 68/68 [00:13<00:00,  5.04it/s]


41 0.8500758175266914 0.07541702137354553


100%|██████████| 68/68 [00:13<00:00,  5.04it/s]


8 0.8558784122418784 0.06674612327521944


100%|██████████| 68/68 [00:13<00:00,  5.02it/s]


75 0.798420417133524 0.08322614569462733


100%|██████████| 68/68 [00:13<00:00,  5.03it/s]


In [ ]:
print(f"Kfold Score : {model_score_sum / len(config['FOLD'])}")

In [11]:
print(f_score_avg.shape)
print(f_score_avg)
f_score_avg = np.array(f_score_avg) / len(config['FOLD'])
f_score_avg = f_score_avg.tolist()

(2154, 88)
[[-20.36523438 -28.6640625  -21.07421875 ... -20.26953125 -30.51367188
  -30.9921875 ]
 [-32.65234375 -33.2265625  -32.10546875 ... -30.60546875 -32.07421875
  -30.86914062]
 [-21.11621094 -24.67382812 -21.3515625  ... -22.42773438 -25.82226562
  -24.90625   ]
 ...
 [-27.8203125  -30.765625   -29.93945312 ... -31.0546875  -30.69921875
  -32.06445312]
 [-26.671875   -28.37890625 -27.234375   ... -27.33203125 -27.4609375
  -27.390625  ]
 [-34.828125   -37.8671875  -34.28125    ... -26.38867188 -22.96777344
  -15.625     ]]


In [12]:
y = np.array(f_score_avg).argmax(1)
y = encoder.inverse_transform(y)
f_pred = y.tolist()

In [14]:
submission = pd.read_csv(config["TEST_CSV"])
submission['label'] = f_pred
submission['score'] = f_score
del submission['file_name']
del submission['path']
submission

,index,label,score
0,0,tile-glue_strip,"[-3.50390625, -5.41015625, -4.28515625, -6.382..."
1,1,grid-good,"[-8.171875, -7.84375, -9.0546875, -7.890625, -..."
2,2,transistor-misplaced,"[-5.47265625, -5.453125, -4.5234375, -6.144531..."
3,3,tile-gray_stroke,"[-3.607421875, -5.96484375, -2.529296875, -6.0..."
4,4,tile-good,"[-7.9609375, -7.31640625, -6.875, -5.89453125,..."
...,...,...,...
2149,2149,tile-gray_stroke,"[-3.134765625, -6.12109375, -2.076171875, -4.9..."
2150,2150,screw-good,"[-8.1484375, -7.3515625, -7.39453125, -7.07421..."
2151,2151,grid-good,"[-6.5859375, -6.6640625, -7.70703125, -6.69921..."
2152,2152,cable-good,"[-6.12890625, -7.1484375, -7.20703125, -5.9179..."


In [15]:
submission.to_csv(config["SUBMISSION_CSV"], index=None)

In [16]:
del submission['score']
submission.to_csv('swin_kfold_submission', index=None)

In [17]:
(0.7421643212630613 + 0.8163526326732292 + 0.8500758175266914 + 0.8558784122418784 + 0.798420417133524) / 5

0.812578320167677